In [60]:
import psycopg2 as pg
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.kernel_approximation import Nystroem, RBFSampler
# from sklearn import svm
from sklearn.svm import LinearSVC
%matplotlib inline

In [61]:
columns_business = [
'business_id',
# 'name',
# 'address',
#'city',
'state',
#'postal_code',
'latitude',
'longitude',
# 'stars',
'review_count',
'attributes_goodforkids',
#'categories',   further processing
'is_open',
# 'hours_monday',
# 'hours_tuesday',
# 'hours_wednesday',
# 'hours_thursday',
# 'hours_friday',
# 'hours_saturday',
# 'hours_sunday',
'attributes_restaurantsreservations',
# 'attributes_goodformeal',
# 'attributes_businessparking',
'attributes_caters',
'attributes_noiselevel',
'attributes_restaurantstableservice',
'attributes_restaurantstakeout',
'attributes_restaurantspricerange2',
'attributes_outdoorseating',
'attributes_bikeparking',
# 'attributes_ambience',
'attributes_hastv',
'attributes_wifi',
'attributes_alcohol',
'attributes_restaurantsattire',
'attributes_restaurantsgoodforgroups',
'attributes_restaurantsdelivery',
'attributes_businessacceptscreditcards',
'attributes_businessacceptsbitcoin',
#'attributes_byappointmentonly',
#'attributes_acceptsinsurance',
# 'attributes_music',
'attributes_goodfordancing',
'attributes_coatcheck',
'attributes_happyhour',
# 'attributes_bestnights',
'attributes_wheelchairaccessible',
'attributes_dogsallowed',
# 'attributes_byobcorkage',
'attributes_drivethru',
'attributes_smoking',
#'attributes_agesallowed',
#'attributes_hairspecializesin',
#'attributes_corkage',
#'attributes_byob',
#'attributes_dietaryrestrictions', further processing
#'attributes_open24hours',
#'attributes_restaurantscounterservice',
'restaurant',
'meanfunny',
'meanuseful',
'avgwordcount',
'maxwordcount',
'minwordcount',
'avgfunnywordcount',
'maxfunnywordcount',
'avgusefulwordcount',
'maxusefulwordcount',
'medianwordcount',
'upperquartilewordcount',
'lowerquartilewordcount',
'target']

In [62]:
list_as_string =', '.join(columns_business)

In [63]:
con = pg.connect(database="postgres", user="flatiron_user_1", password="K*%4t3VK0ab%gn", host="34.74.239.44", port="5432")
cur = con.cursor()

u_cutoff=87
l_cutoff=10

In [64]:

cur.execute(f"SELECT {list_as_string} FROM business WHERE restaurant IS true AND REVIEW_COUNT < {u_cutoff} AND REVIEW_COUNT > {l_cutoff}")
dataframe=cur.fetchall()

In [65]:
dataframe=pd.DataFrame(dataframe)
dataframe.columns=columns_business


In [66]:
dataframe.target.value_counts()

1    24234
0    13384
Name: target, dtype: int64

In [67]:
np.mean(dataframe.isnull(), axis=0)

business_id                              0.000000
state                                    0.000000
latitude                                 0.000000
longitude                                0.000000
review_count                             0.000000
attributes_goodforkids                   0.259743
is_open                                  0.000000
attributes_restaurantsreservations       0.268462
attributes_caters                        0.414482
attributes_noiselevel                    0.350843
attributes_restaurantstableservice       0.761391
attributes_restaurantstakeout            0.153357
attributes_restaurantspricerange2        0.050906
attributes_outdoorseating                0.218061
attributes_bikeparking                   0.213781
attributes_hastv                         0.290473
attributes_wifi                          0.318784
attributes_alcohol                       0.298474
attributes_restaurantsattire             0.314743
attributes_restaurantsgoodforgroups      0.239965


In [68]:
dataframe.nunique()

business_id                              37618
state                                       11
latitude                                 34993
longitude                                34729
review_count                                76
attributes_goodforkids                       3
is_open                                      2
attributes_restaurantsreservations           3
attributes_caters                            3
attributes_noiselevel                        9
attributes_restaurantstableservice           3
attributes_restaurantstakeout                3
attributes_restaurantspricerange2            5
attributes_outdoorseating                    3
attributes_bikeparking                       3
attributes_hastv                             3
attributes_wifi                              7
attributes_alcohol                           7
attributes_restaurantsattire                 7
attributes_restaurantsgoodforgroups          3
attributes_restaurantsdelivery               3
attributes_bu

In [69]:
dataframe.attributes_restaurantsreservations.value_counts()
dataframe.attributes_smoking.value_counts()

u'no'         762
u'outdoor'    636
u'yes'        173
'no'            8
None            4
'outdoor'       4
'yes'           1
Name: attributes_smoking, dtype: int64

In [70]:
list_to_strip=[
    'attributes_alcohol',
    'attributes_restaurantsattire',
    'attributes_wifi',
    'attributes_smoking',
    'attributes_noiselevel',
              ]
dataframe2=dataframe.copy()
for col in list_to_strip:
    dataframe2[col]=dataframe2[col].str.strip("u\'")

In [71]:
for col in columns_business:
    dataframe2[col]=dataframe2[col].where(dataframe2[col]!='None')
    dataframe2[col]=dataframe2[col].where(dataframe2[col]!='none')


/home/will/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [72]:
dataframe2.attributes_restaurantstakeout.nunique(dropna=False)

4

In [73]:
dataframe2.attributes_restaurantstakeout.value_counts(dropna=False)


True     29382
NaN       5786
False     2450
Name: attributes_restaurantstakeout, dtype: int64

In [74]:
dataframe2.head()

,business_id,state,latitude,longitude,review_count,attributes_goodforkids,is_open,attributes_restaurantsreservations,attributes_caters,attributes_noiselevel,...,maxwordcount,minwordcount,avgfunnywordcount,maxfunnywordcount,avgusefulwordcount,maxusefulwordcount,medianwordcount,upperquartilewordcount,lowerquartilewordcount,target
0,8i6BaUxYwaewz4nj9R2ppQ,AZ,33.4960409,-112.0205662,51,None,1,None,None,None,...,308.0,13.0,69.0,161.0,106.0,308.0,57.5,83.25,38.0,1
1,igo--IP_NBWIO_yGtBJ0Eg,ON,43.764552,-79.4100088,11,None,1,None,None,None,...,369.0,14.0,50.0,51.0,72.0,129.0,109.0,138.00,50.0,1
2,kzh7iDNd8Qwio2w3biAHGw,NV,36.1214517,-115.1696112,22,False,1,None,None,None,...,770.0,110.0,248.0,514.0,309.0,770.0,224.5,410.50,172.5,1
3,q6dOVAsEZ0_JeVcuq1htIw,OH,41.3798611,-82.2227923,19,None,1,None,None,None,...,232.0,14.0,121.0,218.0,57.0,57.0,58.5,96.50,44.5,1
4,dGrZ45A7XZ-fjfl4JrG77A,AB,51.0958026,-114.14072,11,None,1,None,None,None,...,289.0,33.0,108.0,133.0,190.0,260.0,155.0,199.50,83.0,1


In [75]:
cols = dataframe.columns
cols = cols.drop(['review_count',
                  'restaurant',
                  'latitude',
                  'longitude',
                  'business_id',
                  'meanfunny',
                  'meanuseful',
                  'avgwordcount',
                  'maxwordcount',
                  'minwordcount',
                  'avgfunnywordcount',
                  'maxfunnywordcount',
                  'avgusefulwordcount',
                  'maxusefulwordcount',
                  'medianwordcount',
                  'upperquartilewordcount',
                  'lowerquartilewordcount',
                  'target'])


In [76]:

dataframe2.latitude=dataframe2.latitude.astype(float)
dataframe2.longitude=dataframe2.longitude.astype(float)

In [77]:
categorical_features=cols
categorical_features

Index(['state', 'attributes_goodforkids', 'is_open',
       'attributes_restaurantsreservations', 'attributes_caters',
       'attributes_noiselevel', 'attributes_restaurantstableservice',
       'attributes_restaurantstakeout', 'attributes_restaurantspricerange2',
       'attributes_outdoorseating', 'attributes_bikeparking',
       'attributes_hastv', 'attributes_wifi', 'attributes_alcohol',
       'attributes_restaurantsattire', 'attributes_restaurantsgoodforgroups',
       'attributes_restaurantsdelivery',
       'attributes_businessacceptscreditcards',
       'attributes_businessacceptsbitcoin', 'attributes_goodfordancing',
       'attributes_coatcheck', 'attributes_happyhour',
       'attributes_wheelchairaccessible', 'attributes_dogsallowed',
       'attributes_drivethru', 'attributes_smoking'],
      dtype='object')

In [78]:
numerical_features=[
    'review_count',
    'latitude',
    'longitude',
    'meanfunny',
    'meanuseful',
    'avgwordcount',
    'maxwordcount',
    'minwordcount',
    'avgfunnywordcount',
    'maxfunnywordcount',
    'avgusefulwordcount',
    'maxusefulwordcount',
    'medianwordcount',
    'upperquartilewordcount',
    'lowerquartilewordcount',
                   ]

In [79]:
preprocess = make_column_transformer(
    (StandardScaler(), numerical_features),
    (OneHotEncoder(drop='first'), categorical_features))

In [80]:
dataframe2[categorical_features]=dataframe2[categorical_features].fillna(9)
dataframe2[numerical_features]=dataframe2[numerical_features].fillna(0)


In [81]:
# preprocess.fit_transform(dataframe).toarray()[:5]
dataframe2[categorical_features]=dataframe2[categorical_features].astype(str)

In [82]:
dataframe2.isna().sum()

business_id                              0
state                                    0
latitude                                 0
longitude                                0
review_count                             0
attributes_goodforkids                   0
is_open                                  0
attributes_restaurantsreservations       0
attributes_caters                        0
attributes_noiselevel                    0
attributes_restaurantstableservice       0
attributes_restaurantstakeout            0
attributes_restaurantspricerange2        0
attributes_outdoorseating                0
attributes_bikeparking                   0
attributes_hastv                         0
attributes_wifi                          0
attributes_alcohol                       0
attributes_restaurantsattire             0
attributes_restaurantsgoodforgroups      0
attributes_restaurantsdelivery           0
attributes_businessacceptscreditcards    0
attributes_businessacceptsbitcoin        0
attributes_

In [83]:
ohe=OneHotEncoder(drop='first')
ohe.fit(dataframe2[categorical_features])

OneHotEncoder(categorical_features=None, categories=None, drop='first',
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [84]:
ohe.transform(dataframe2[categorical_features])

<37618x66 sparse matrix of type '<class 'numpy.float64'>'
	with 479656 stored elements in Compressed Sparse Row format>

In [85]:
dataframe2[categorical_features].head()

,state,attributes_goodforkids,is_open,attributes_restaurantsreservations,attributes_caters,attributes_noiselevel,attributes_restaurantstableservice,attributes_restaurantstakeout,attributes_restaurantspricerange2,attributes_outdoorseating,...,attributes_restaurantsdelivery,attributes_businessacceptscreditcards,attributes_businessacceptsbitcoin,attributes_goodfordancing,attributes_coatcheck,attributes_happyhour,attributes_wheelchairaccessible,attributes_dogsallowed,attributes_drivethru,attributes_smoking
0,AZ,9,1,9,9,9,9,9,1,9,...,9,True,9,9,9,9,True,9,9,9
1,ON,9,1,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2,NV,False,1,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
3,OH,9,1,9,9,9,9,9,9,9,...,9,True,9,9,9,9,9,9,9,9
4,AB,9,1,9,9,9,9,9,3,9,...,9,9,9,9,9,9,9,9,9,9


In [86]:
preprocess = make_column_transformer(
    (StandardScaler(), numerical_features),
    (OneHotEncoder(drop='first'), categorical_features)
    
)
# preprocess.fit_transform(dataframe2)

In [175]:
pipe = Pipeline([('scl', preprocess),
                 ('clf', RandomForestClassifier(oob_score=True,n_jobs=-1,verbose=1))])

param_range = [20,25,30]

grid_params_rf = [{'clf__criterion': ['gini'],
                   'clf__n_estimators': [300,500],
                   'clf__min_samples_leaf': [10,13,15,17],
                   'clf__max_depth': [20,25,30],
                   #'clf__min_samples_split': [1,100]
                   'clf__max_features' : ['log2','sqrt']
                   
                   }]


In [171]:
# pipe = Pipeline([('ct', preprocess),
#                  ('clf', LogisticRegression())])

# grid_params_lr = [{'clf__penalty': ['l1', 'l2'],
#                    'clf__C': [.001,.01,.1,1,10,100,1000],
#                   }] 


In [137]:
pipe_svm = Pipeline([('ct', preprocess),
                    ("feature_map", Nystroem(gamma=.2)),
                    ('clf', LinearSVC(verbose=1, max_iter=15000, dual=False))])

param_range = [3000,5000,7000]

grid_params_svm = [{'feature_map__gamma' : [.005,.01,.1],
                    'clf__C': param_range,
                    'clf__penalty': ['l1']
                   }]

In [162]:
pipe_svm = Pipeline([('ct', preprocess),
                    ("feature_map", RBFSampler()),
                    ('clf', LinearSVC(verbose=1, max_iter=15000, dual=False))])

param_range = [1000,2000,3000]

grid_params_svm = [{'feature_map__gamma' : [.005,.01,.02],
                    'clf__C': param_range,
                    'clf__penalty': ['l1']
                   }]

In [176]:
gridsearch = GridSearchCV(estimator=pipe,
			param_grid=grid_params_rf,
			scoring='accuracy',
            return_train_score=True,
			cv=5)

In [173]:

data=dataframe2.drop(['business_id','restaurant','target'],axis=1)
target=dataframe2.target
data.columns

Index(['state', 'latitude', 'longitude', 'review_count',
       'attributes_goodforkids', 'is_open',
       'attributes_restaurantsreservations', 'attributes_caters',
       'attributes_noiselevel', 'attributes_restaurantstableservice',
       'attributes_restaurantstakeout', 'attributes_restaurantspricerange2',
       'attributes_outdoorseating', 'attributes_bikeparking',
       'attributes_hastv', 'attributes_wifi', 'attributes_alcohol',
       'attributes_restaurantsattire', 'attributes_restaurantsgoodforgroups',
       'attributes_restaurantsdelivery',
       'attributes_businessacceptscreditcards',
       'attributes_businessacceptsbitcoin', 'attributes_goodfordancing',
       'attributes_coatcheck', 'attributes_happyhour',
       'attributes_wheelchairaccessible', 'attributes_dogsallowed',
       'attributes_drivethru', 'attributes_smoking', 'meanfunny', 'meanuseful',
       'avgwordcount', 'maxwordcount', 'minwordcount', 'avgfunnywordcount',
       'maxfunnywordcount', 'avgusefu

In [174]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=50)

In [177]:
gridsearch.fit(X_train, y_train);

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 293 out of 300 | elapsed:    5.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    5.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 212 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 

In [179]:
cross_val = pd.DataFrame(gridsearch.cv_results_)
cross_val2 = cross_val.sort_values('rank_test_score')
cross_val2.head(20)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__criterion,param_clf__max_depth,param_clf__max_features,param_clf__min_samples_leaf,param_clf__n_estimators,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
40,6.973171,0.058901,0.271988,0.001037,gini,30,sqrt,10,300,"{'clf__criterion': 'gini', 'clf__max_depth': 3...",...,0.717585,0.003375,1,0.833264,0.834884,0.833984,0.833195,0.833361,0.833738,0.000638
8,7.960148,0.626817,0.326094,0.052574,gini,20,sqrt,10,300,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",...,0.716721,0.002452,2,0.829899,0.830398,0.830952,0.831492,0.830329,0.830614,0.000552
25,11.770920,0.524772,0.370301,0.001633,gini,25,sqrt,10,500,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",...,0.716621,0.002780,3,0.831021,0.835133,0.832696,0.834607,0.833569,0.833405,0.001459
41,11.293772,0.071876,0.369215,0.000919,gini,30,sqrt,10,500,"{'clf__criterion': 'gini', 'clf__max_depth': 3...",...,0.716588,0.003250,4,0.831810,0.833887,0.834109,0.833776,0.833153,0.833347,0.000831
9,13.301188,1.061143,0.405333,0.036187,gini,20,sqrt,10,500,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",...,0.716522,0.003647,5,0.829899,0.832143,0.830495,0.831284,0.829997,0.830764,0.000846
24,7.068337,0.099294,0.276958,0.010051,gini,25,sqrt,10,300,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",...,0.716422,0.004056,6,0.831561,0.834926,0.832987,0.833569,0.832572,0.833123,0.001114
43,11.573506,0.795402,0.466995,0.182361,gini,30,sqrt,13,500,"{'clf__criterion': 'gini', 'clf__max_depth': 3...",...,0.715890,0.003548,7,0.808549,0.809379,0.809146,0.808772,0.810060,0.809181,0.000525
10,7.286271,0.808188,0.287425,0.028515,gini,20,sqrt,13,300,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",...,0.715691,0.002940,8,0.805766,0.807053,0.806779,0.806529,0.805906,0.806407,0.000497
27,13.989273,0.803375,0.544169,0.051653,gini,25,sqrt,13,500,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",...,0.715691,0.003454,8,0.807801,0.809296,0.808315,0.808398,0.807568,0.808276,0.000597
26,8.173499,0.409211,0.337586,0.113276,gini,25,sqrt,13,300,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",...,0.715159,0.002939,10,0.808549,0.809006,0.808108,0.807651,0.807526,0.808168,0.000553


In [169]:
cross_val2.to_pickle('SVM_MC_RBF_3_CV')

In [168]:
cross_val3=pd.read_pickle('Random_Forest_2_CV')
cross_val3.head(20)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__criterion,param_clf__max_depth,param_clf__min_samples_leaf,param_clf__n_estimators,params,split0_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,4.899049,0.667398,0.261337,0.077889,gini,25,13,150,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",0.720598,...,0.715059,0.002817,1,0.807302,0.808341,0.809188,0.807526,0.807443,0.807960,0.000713
7,4.913177,0.123141,0.266882,0.043027,gini,30,15,150,"{'clf__criterion': 'gini', 'clf__max_depth': 3...",0.718439,...,0.714628,0.002569,2,0.794010,0.795879,0.795813,0.796561,0.795730,0.795599,0.000848
0,5.205843,0.721770,0.239730,0.074568,gini,20,13,150,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",0.717276,...,0.714328,0.002294,3,0.802692,0.807053,0.805948,0.806654,0.806363,0.805742,0.001567
5,5.007015,1.037283,0.311627,0.089086,gini,25,17,150,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",0.715615,...,0.714129,0.001852,4,0.785453,0.786741,0.786759,0.786925,0.787299,0.786635,0.000624
6,5.272803,0.424887,0.291921,0.014758,gini,30,13,150,"{'clf__criterion': 'gini', 'clf__max_depth': 3...",0.719269,...,0.713996,0.003539,5,0.806222,0.808466,0.809437,0.806446,0.807651,0.807644,0.001212
2,5.298776,0.734458,0.315583,0.076946,gini,20,17,150,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",0.715282,...,0.713631,0.001459,6,0.785079,0.786616,0.785554,0.786302,0.784931,0.785696,0.000663
4,4.749132,0.806110,0.260844,0.105720,gini,25,15,150,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",0.719934,...,0.713464,0.003454,7,0.793096,0.797416,0.795439,0.795439,0.795024,0.795283,0.001375
1,5.691518,0.932559,0.268816,0.076168,gini,20,15,150,"{'clf__criterion': 'gini', 'clf__max_depth': 2...",0.716777,...,0.713298,0.002241,8,0.794550,0.794342,0.795398,0.794650,0.795315,0.794851,0.000425
8,5.008572,0.128928,0.285968,0.001116,gini,30,17,150,"{'clf__criterion': 'gini', 'clf__max_depth': 3...",0.716113,...,0.712733,0.002513,9,0.785370,0.787198,0.787423,0.784516,0.788462,0.786594,0.001439
